In [1]:
!pip install "holidays==0.24"
import holidays

!pip install "prophet==1.1.1"
from prophet import Prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: holidays
    Found existing installation: holidays 0.50
    Uninstalling holidays-0.50:
      Successfully uninstalled holidays-0.50
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.5 requires holidays>=0.25, but you have holidays 0.24 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.4 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732001 sha256=c4681f3e0cb7319e19bf4f330c295418bc39baebf6c90aa8fd

In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [3]:
#df = pd.read_excel('Ingresos_2.xlsx')
#df['Fecha '] = pd.to_datetime(df['Fecha '])
#df.rename(columns={'Fecha ': 'Fecha'}, inplace=True)
#df.drop_duplicates(subset='Fecha', keep='last', inplace=True)

In [4]:
def fit_prophet_model(data, column, dias):
  print(column)
  if column != 'hombres':
      data[column] += np.random.normal(0, data[column].std() * 0.1, size=len(data))
      df_prophet = data.reset_index().rename(columns={'fecha': 'ds', column: 'y'})

      cap_value = df_prophet['y'].max() * 1.5
      df_prophet['cap'] = cap_value

      model = Prophet(
          growth="logistic",
          seasonality_mode='multiplicative',
          changepoint_prior_scale=0.5,
          interval_width=0.95
      )
      model.fit(df_prophet)
      future = model.make_future_dataframe(periods=dias)
      future['cap'] = df_prophet['cap'].iloc[0]
      forecast = model.predict(future)
  elif column == 'hombres':
      data[column] += np.random.normal(0, data[column].std() * 0.1, size=len(data))
      df_prophet = data.reset_index().rename(columns={'fecha': 'ds', column: 'y'})

      model = Prophet(
          growth="linear")
      model.fit(df_prophet)
      future = model.make_future_dataframe(periods=dias)
      forecast = model.predict(future)

  forecast = forecast[['ds', 'yhat']]
  return model, forecast

def create_models(df, dias):
  forecasts = {}
  # Entrenar un modelo para cada columna
  for column in df.columns[1:-1]:
      model, forecast = fit_prophet_model(df, column, dias)
      forecasts[column] = forecast

  total_forecast = pd.DataFrame({'ds': forecasts[list(forecasts.keys())[0]]['ds']})
  total_forecast['yhat'] = sum(forecast['yhat'].round().astype(int) for forecast in forecasts.values())
  forecasts['total'] = total_forecast

  #for column in df.columns[1:]:
  #  forecast = forecasts[column]
  return forecasts

In [5]:
df = pd.read_excel('ingresos.xlsx') ########## aca seria el fetch decrypt
df.columns = ['fecha', 'hombres', 'mujeres', 'niños', 'niñas', 'lgbtqi', 'total']
forecasts = create_models(df, 5)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/_muvhcql.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/3x6zxz8m.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97444', 'data', 'file=/tmp/tmph6kprig1/_muvhcql.json', 'init=/tmp/tmph6kprig1/3x6zxz8m.json', 'output', 'file=/tmp/tmph6kprig1/prophet_modeltr6z0z3h/prophet_model-20240611235229.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:52:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


hombres


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/z5inps6o.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/nrzv1nz3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=21489', 'data', 'file=/tmp/tmph6kprig1/z5inps6o.json', 'init=/tmp/tmph6kprig1/nrzv1nz3.json', 'output', 'file=/tmp/tmph6kprig1/prophet_model7c0f8gfg/prophet_model-20240611235230.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:52:30 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


mujeres


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/kuawk89k.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/pxvtj1qq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12710', 'data', 'file=/tmp/tmph6kprig1/kuawk89k.json', 'init=/tmp/tmph6kprig1/pxvtj1qq.json', 'output', 'file=/tmp/tmph6kprig1/prophet_modelfj9kkqba/prophet_model-20240611235231.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:52:31 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/3h3lgjpg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/8x_ddsq6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3359', 'data', 'file=/tmp/tmph6kprig1/3h3lgjpg.json', 'init=/tmp/tmph6kprig1/8x_ddsq6.json', 'output', 'file=/tmp/tmph6kprig1/prophet_modelppzime56/prophet_model-20240611235233.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


niñas


23:52:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/ebaqdnv_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph6kprig1/_fma68_a.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=44302', 'data', 'file=/tmp/tmph6kprig1/ebaqdnv_.json', 'init=/tmp/tmph6kprig1/_fma68_a.json', 'output', 'file=/tmp/tmph6kprig1/prophet_modeltl_v20x6/prophet_model-20240611235234.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:52:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


lgbtqi


23:52:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [6]:
# Ingresos y Predicciones juntos
colors = {
    'hombres': 'blue',
    'mujeres': 'hotpink',
    'niños': 'deepskyblue',
    'niñas': 'deeppink',
    'lgbtqi': 'red',
    'total': 'orange'
}

# Create a figure
fig = go.Figure()

# Graficar las predicciones
#plt.figure(figsize=(16, 8), dpi=150)
for column in df.columns[1:]:
    forecast = forecasts[column]
    # Graficar predicciones con línea punteada
    fig.add_trace(go.Scatter(x = forecast['ds'], y = forecast['yhat'].round().astype(int), mode='lines', name=f'Predicción {column}', line=dict(dash='dot', color=colors[column])))
    # Graficar valores reales con línea sólida
    fig.add_trace(go.Scatter(x = df['fecha'], y = df[column].round().astype(int), mode='lines', name=f'Real {column}', line=dict(dash='solid', color = colors[column])))

# Add vertical line
fig.add_shape(
    type='line',
    x0=df['fecha'].iloc[-1],
    y0=0,
    x1=df['fecha'].iloc[-1],
    y1=1,
    xref='x',
    yref='paper',
    line=dict(color='black', dash='dash'),
    name='Inicio de la predicción'
)

# Agregar una línea vertical invisible para la leyenda
fig.add_trace(go.Scatter(
    x=[df['fecha'].iloc[-1], df['fecha'].iloc[-1]],
    y=[0, 1],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Inicio de la predicción'
))

fig.update_layout(
    title='Evolución de ingresos',
    xaxis_title='Días',
    yaxis_title='Ingresos',
    legend_title='Leyenda'
)

# Agregar características interactivas
fig.update_traces(mode='lines')

# Guardar como archivo HTML
fig.write_html("evolucion_ingresos.html")

fig.show()

In [1]:
import pandas as pd
df = pd.read_excel("ingresos.xlsx")